In [1]:
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.tag import pos_tag
data1_path = "YOUR_DATA_PATH"
data2_path = "YOUR_DATA_PATH"
df_data1 = pd.read_csv(data1_path, names=["ENGLISH", "CATEGORY"])
df_data1
df_data2 = pd.read_csv(data2_path, encoding="latin")
# Drop the ORIGINAL column in the dataframe
df_data2.drop("ORIGINAL", axis=1, inplace=True)
df = pd.concat([df_data1, df_data2])
# ---- YOU CAN CHANGE WITH YOUR OWN COLUMN NAMES
df.CATEGORY.replace(['Trouble', 'Troubleshooting'], 'Troubleshooting', inplace=True)
df.CATEGORY.replace(['User Manual', ' User Manual'], 'User Manual', inplace=True)
df.CATEGORY.replace(['Product Match', 'Product', 'Product Info', 'Product '], 'Product', inplace=True)
df.CATEGORY.replace([' split & Cook '], 'Split & Cook', inplace=True)
df.CATEGORY.replace(['Usage '], 'Usage', inplace=True)
df.CATEGORY.replace(['Website '], 'Website', inplace=True)
df.CATEGORY.replace(['Customer Service', 'Technical Service'], 'Customer & Technical Service', inplace=True)
df.drop(df[df.CATEGORY == "Program"].index, inplace=True)
df.drop(df[df.CATEGORY == "CATEGORY"].index, inplace=True)
df.CATEGORY.value_counts()
# Reset indexing to start from 0
df.reset_index(drop=True, inplace=True)

product_code = pd.read_excel("UrunOzellikleri_v2.xlsx")
product_code_set = set(product_code.iloc[:, 0])
typo_dict = pd.read_csv("typo_dictionary.csv", sep=": ")
typo_dict.FOR = typo_dict.FOR.str.replace("'", "")
typo_dict.EN = typo_dict.EN.str.replace("'", "")
typo_dict.EN = typo_dict.EN.str.replace(",", "")
# Convert to Python dict FOR as a key and EN as a value
typos = {typo_dict.FOR[i]: typo_dict.EN[i] for i in range(len(typo_dict))}
df.CATEGORY.value_counts()

/tmp/ipykernel_259/3011291259.py:37: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  typo_dict = pd.read_csv("typo_dictionary.csv", sep=": ")


Product                         2505
User Manual                      479
Registration                     250
Website                          246
Customer & Technical Service     216
Accessories & Spare Parts        203
Troubleshooting                  174
Usage                            151
Other                            133
Installation                     110
Name: CATEGORY, dtype: int64

In [2]:
def preprocess_text(text):
    text = str(text)
    words = word_tokenize(text)
    text = " ".join(words)
    print(text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    # write a regex pattern if the word is include number or numbers
    words = re.sub(r"[a-zA-Z0-9]*[0-9]+[a-zA-Z0-9]*", "", text).split()
    words = [word for word in words if word not in product_code_set]
    st = stopwords.words('english')
    words = [pos_tag([word])[0][0] if pos_tag([word])[0][1].startswith("V") else word for word in words]
    words = [word for word in words if word not in [' ', '', '\n']]
    words = [word.lower() for word in words if len(word) > 2]
    words = [word for word in words if word not in st]
    words = [typos[word] if word in typos else word for word in words]
    return " ".join(words)

print(preprocess_text("The appearance of the outdoor unit BAH 120/121"))

The appearance of the outdoor unit BAH 120/121
appearance outdoor unit bah


In [ ]:
# Preprocess the ENGLISH column
df["PREPROCESS"] = df["ENGLISH"].apply(preprocess_text)
# Change column order CATEGORY and PREPROCESS
df = df[["ENGLISH", "PREPROCESS", "CATEGORY"]]
df

In [ ]:
df = df[df.PREPROCESS != ""]
# Count all the word in the PREPROCESS column
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df["PREPROCESS"])
count_vect.get_feature_names()
frequency_dict = {word: X_counts.getcol(count_vect.vocabulary_[word]).sum() for word in count_vect.get_feature_names()}
frequency_dict = sorted(frequency_dict.items(), key=lambda x: x[1], reverse=False)
frequency_ones = [word[0] for word in frequency_dict if word[1] == 1]
len(frequency_ones)
def remove_one_word_from_text(text):
    if "ï¿½" in text:
        text = text.replace("ï¿½", "")
    words = word_tokenize(text)
    words = [word for word in words if word not in frequency_ones]
    return " ".join(words)
    
df["PREPROCESS"] = df["PREPROCESS"].apply(remove_one_word_from_text)



In [6]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df["PREPROCESS"])
count_vect.get_feature_names()
frequency_dict = {word: X_counts.getcol(count_vect.vocabulary_[word]).sum() for word in count_vect.get_feature_names()}
frequency_dict = sorted(frequency_dict.items(), key=lambda x: x[1], reverse=False)
frequency_dict = dict(frequency_dict)
frequency_dict


{'after': 1,
 'antonio': 1,
 'blowing': 1,
 'both': 1,
 'but': 1,
 'bva': 1,
 'corteseente': 1,
 'expansinnos': 1,
 'fabionincheri': 1,
 'fichaenergtica': 1,
 'for': 1,
 'gabriella': 1,
 'gianni': 1,
 'herbert': 1,
 'how': 1,
 'if': 1,
 'ingls': 1,
 'lorenzo': 1,
 'lung': 1,
 'marco': 1,
 'my': 1,
 'nellla': 1,
 'no': 1,
 'nprodotto': 1,
 'pc': 1,
 'pedrea': 1,
 'posiblecolaboracin': 1,
 'push': 1,
 'shame': 1,
 'them': 1,
 'who': 1,
 'your': 1,
 'ability': 2,
 'absence': 2,
 'accessories': 2,
 'according': 2,
 'acquastop': 2,
 'acquired': 2,
 'actually': 2,
 'addition': 2,
 'ads': 2,
 'advice': 2,
 'afternoon': 2,
 'alarm': 2,
 'almost': 2,
 'among': 2,
 'anti': 2,
 'anymore': 2,
 'anything': 2,
 'apply': 2,
 'appropriate': 2,
 'around': 2,
 'article': 2,
 'aspirator': 2,
 'assurance': 2,
 'auto': 2,
 'automatic': 2,
 'baby': 2,
 'background': 2,
 'bagless': 2,
 'bah': 2,
 'baking': 2,
 'bearprimo': 2,
 'bgms': 2,
 'bill': 2,
 'bit': 2,
 'block': 2,
 'bologna': 2,
 'brief': 2,
 'bring

In [7]:
df.to_csv("YOUR_CSV_PATH", index=False)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.PREPROCESS, df.CATEGORY, test_size=0.025, random_state=42)

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=150)),
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=150))])

In [316]:
pipeline.score(X_train, y_train)
pipeline.score(X_test, y_test)


0.8712871287128713

In [10]:
# Show confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = pipeline.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 3,  0,  0,  0,  2,  0,  0,  0,  0,  0],
       [ 0,  5,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 48,  1,  0,  1,  1,  1],
       [ 0,  0,  0,  0,  0,  8,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 16,  0],
       [ 0,  1,  0,  0,  1,  0,  0,  0,  0,  5]])

In [13]:
# classfication report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


                              precision    recall  f1-score   support

   Accessories & Spare Parts       1.00      0.60      0.75         5
Customer & Technical Service       0.71      0.83      0.77         6
                Installation       1.00      1.00      1.00         1
                       Other       0.00      0.00      0.00         2
                     Product       0.86      0.92      0.89        52
                Registration       0.89      1.00      0.94         8
             Troubleshooting       0.00      0.00      0.00         2
                       Usage       0.50      0.50      0.50         2
                 User Manual       0.94      1.00      0.97        16
                     Website       0.83      0.71      0.77         7

                    accuracy                           0.86       101
                   macro avg       0.67      0.66      0.66       101
                weighted avg       0.83      0.86      0.84       101



/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [322]:
# upload test file
test_file = pd.read_excel("YOUR_TEST_FILE_PATH")

In [ ]:
# Add new column PREPROCESS
test_file["PREPROCESS"] = ""

test_file["PREPROCESS"] = test_file["Translated Comment"].apply(preprocess_text)

In [ ]:
test_file = test_file[test_file.PREPROCESS != ""]
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(test_file["PREPROCESS"])
count_vect.get_feature_names()
frequency_dict = {word: X_counts.getcol(count_vect.vocabulary_[word]).sum() for word in count_vect.get_feature_names()}
frequency_dict = sorted(frequency_dict.items(), key=lambda x: x[1], reverse=False)
frequency_ones = [word[0] for word in frequency_dict if word[1] == 1]
len(frequency_ones)
def remove_one_word_from_text(text):
    if "ï¿½" in text:
        text = text.replace("ï¿½", "")
    words = word_tokenize(text)
    words = [word for word in words if word not in frequency_ones]
    return " ".join(words)
    
test_file["PREPROCESS"] = test_file["PREPROCESS"].apply(remove_one_word_from_text)

In [343]:
test_file["Category"] = ""


In [ ]:

test_file["Category"] = pipeline.predict(test_file.PREPROCESS)
# Remove PREPROCESS column
test_file = test_file.drop(["PREPROCESS"], axis=1)
test_file

In [348]:
test_file.to_excel("YOUR_FINAL_PREDICT", index=False)